In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [4]:
# Load Encoder and Scaler

with open('le_gender.pkl','rb') as f:
    le_gender=pickle.load(f)

with open('ohe_geo.pkl','rb') as f:
    ohe_geo= pickle.load(f)

with open('std_scaler.pkl','rb') as f:
    scaler=pickle.load(f)

In [5]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000.0
}


In [6]:
# Convert input data to data frame
input_df= pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Female,40,3,60000.0,2,1,1,50000.0


In [7]:
# Transform Gender Column

input_df['Gender']= le_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000.0,2,1,1,50000.0


In [8]:
# Transform Geography Column

geo_transformed= ohe_geo.transform(input_df[['Geography']])
geo_transformed

array([[1., 0., 0.]])

In [9]:
geo_transformed_features_name= ohe_geo.get_feature_names_out(['Geography'])
geo_transformed_features_name

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_transformed_df= pd.DataFrame(geo_transformed,columns=geo_transformed_features_name)
geo_transformed_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
# Drop original Geography Column and concatenate transformed geography column

input_df= input_df.drop('Geography', axis=1)
input_df=pd.concat([input_df.reset_index(drop=True),geo_transformed_df], axis=1)

In [12]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000.0,2,1,1,50000.0,1.0,0.0,0.0


In [13]:
# Apply Standard Scaler

input_scaled= scaler.transform(input_df)
input_scaled

array([[-0.51795799, -1.10275404,  0.10698342, -0.69673114, -0.26352876,
         0.79286681,  0.64298333,  0.97067965, -0.86629511,  1.002002  ,
        -0.57946723, -0.57677292]])

In [14]:
# Load Model

model=load_model('model.h5')

In [15]:
# Make Predictions

prediction= model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 97ms/step


array([[0.14252228]], dtype=float32)

In [16]:
prediction_probability= prediction[0][0]
prediction_probability

0.14252228

In [17]:
if prediction_probability>0.5:
    print('The customer is likely to churn.')
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
